<a href="https://colab.research.google.com/github/haowei-2014/llama2/blob/main/llama2_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer
import transformers
import torch
from langchain import HuggingFacePipeline

In [4]:

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1000
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Sure, here are some other shows that are similar to "Breaking Bad" and "Band of Brothers":

1. "The Sopranos" - This HBO series is about a New Jersey mob boss and his family. It's known for its complex characters, gripping storylines, and sharp dialogue.
2. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the notorious Colombian drug lord. It's a gritty and intense drama that explores the drug trade and its impact on society.
3. "Peaky Blinders" - This BBC series is set in post-World War I England and follows a gangster family as they attempt to maintain their power and status in the face of rival


In [5]:
text = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

You are a natural language understanding expert.
Your task is to classify intent and extract slots from a text.

Below are intents and explanations.
- ask_weather: ask for general weather information
- ask_temperature: ask for temperature information
- check_hot: check if the weather is hot
- check_warm: check if the weather is warm
- check_cold: check if the weather is cold
- check_cool: check if the weather is cool
- ask_climate_phenomenon: ask if a specifc climate phenomenon happens, e.g., cloudy, foggy, hail, icy, rain, sand_storm, smog, snow, sunny, windy, frosty


Below are slots and possible values of slots.
- datetime. Possbile values include:
    - morning
    - afternoon
    - night
    - 3 AM
    - 5 PM
    - next 7 days
    - next 4 hours
    - this week
    - Wednesday
- climatic_event. Possbile values include:
    - cloudy
    - foggy
    - hail
    - icy
    - rain
    - sand_storm
    - smog
    - snow
    - sunny
    - windy
    - frosty
- city. Possbile values include:
    - Berlin
    - London
    - Paris
- relative_location. Possible values include:
    - here
    - there
    - destination
    - my destination

Based on the information above, please classify intent and extract slots from the following text.
Regarding slot, only return slot value that is from the text. Do not return irrelevant slot.
Return the result in a json format. Only a json output is needed.

Below is an example.
Text: Weather Berlin next 7 days
Result:
{
    intent: ask_weather
    slot: {
        datetime: next 7 days
        city: Berlin
    }
}

Now, classify intent and extract slots from the below text delimited by triple backquotes.
Return the result in a json format. Only a json output is needed. No explanation or comment is needed.

```Weather forecast in the weekend in Shanghai```

 [/INST]

"""


In [6]:
sequences = pipeline(
    text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=1000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

You are a natural language understanding expert.
Your task is to classify intent and extract slots from a text.

Below are intents and explanations.
- ask_weather: ask for general weather information
- ask_temperature: ask for temperature information
- check_hot: check if the weather is hot
- check_warm: check if the weather is warm
- check_cold: check if the weather is cold
- check_cool: check if the weather is cool
- ask_climate_phenomenon: ask if 

In [7]:
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline)

In [8]:
from langchain import PromptTemplate,  LLMChain

template = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

You are a natural language understanding expert.
Your task is to classify intent and extract slots from a text.

Below are intents and explanations.
- ask_weather: ask for general weather information
- ask_temperature: ask for temperature information
- check_hot: check if the weather is hot
- check_warm: check if the weather is warm
- check_cold: check if the weather is cold
- check_cool: check if the weather is cool
- ask_climate_phenomenon: ask if a specifc climate phenomenon happens, e.g., cloudy, foggy, hail, icy, rain, sand_storm, smog, snow, sunny, windy, frosty


Below are slots and possible values of slots.
- datetime. Possbile values include:
    - morning
    - afternoon
    - night
    - 3 AM
    - 5 PM
    - next 7 days
    - next 4 hours
    - this week
    - Wednesday
- climatic_event. Possbile values include:
    - cloudy
    - foggy
    - hail
    - icy
    - rain
    - sand_storm
    - smog
    - snow
    - sunny
    - windy
    - frosty
- city. Possbile values include:
    - Berlin
    - London
    - Paris
- relative_location. Possible values include:
    - here
    - there
    - destination
    - my destination

Based on the information above, please classify intent and extract slots from the following text.
Regarding slot, only return slot value that is from the text. Do not return irrelevant slot.
Return the result in a json format. Only a json output is needed.

Below are three examples.
Example 1.
Text: Weather Berlin next 7 days
Result:
{{
    intent: ask_weather
    slot: {{
        datetime: next 7 days
        city: Berlin
    }}
}}

Example 2.
Text: Will it rain today
Result:
{{
    intent: check_climate_phenomenon
    slot: {{
        datetime: today
        climatic_event: rain
    }}
}}

Example 3.
Text: Will it be hot tomorrow at my destination
Result:
{{
    intent: check_hot
    slot: {{
        datetime: tomorrow
        relative_location: my destination
    }}
}}


Now, classify intent and extract slots from the below text delimited by triple backquotes.
Return the result in a json format. Only a json output is needed. No explanation or comment is needed.

```{text}```

 [/INST]

"""

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """
Check weather forecast in Spandau in the weekend
"""

print(llm_chain.run(text))

{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}


In [9]:
import time

start = time.time()

for i in range(100):
  print(llm_chain.run(text))

end = time.time()
print(end - start)


{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "intent": "ask_weather",
    "slot": {
        "datetime": "weekend",
        "city": "Spandau"
    }
}
{
    "int

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-13b-chat-hf"

# tokenizer = AutoTokenizer.from_pretrained(model)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# sequences = pipeline(
#     'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=200,
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")



In [ ]:
from langchain import HuggingFacePipeline

# pipeline = transformers.pipeline(
#     "text-generation", #task
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
#     max_length=1000,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id
# )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024
)

llm = HuggingFacePipeline(pipeline = pipeline)

In [ ]:
text = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

  You are a natural language understanding expert.

  We have texts belonging to several domains. Below are the domain names and their explanations.
  - device_control: for example, open/close the window, open/close AC
  - media: for example, play a music or a song
  - navigation: for example, navigate to Berlin
  - phone_call: for example, call my colleague
  - weather: for example, how is the weather today, or will it rainy tomorrow?

  Below is a text delimited by triple backquotes. Your task is to tell me whether it belongs to the weather domain.

                ```Tell me the temperature of Beijing today afternoon```

  Return only "yes" or "no". Do not return anything else.
  Your answer:
 [/INST]

"""

sequences = pipeline(
    text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:

sequences = pipeline(
    text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
text = """

You are a natural language understanding expert.

You task is to tell me whether or not a given text is talking about weather.

Text: What is the weather today in Paris

Answer only yes or no.

"""


# You are a natural language understanding expert.

# We have texts belonging to several domains. Below are the domain names and their explanations.
# - device_control: for example, open/close the window, open/close AC
# - media: for example, play a music or a song
# - navigation: for example, navigate to Berlin
# - phone_call: for example, call my colleague
# - weather: for example, how is the weather today, or will it rainy tomorrow?

# You task is to tell me whether or not a given text belongs to the weather domain.

# Text: What is the weather today in Paris

# Answer only yes or no.

# Tranlate the following text from English to Chinese

# What is the weather today in Paris

In [ ]:
sequences = pipeline(
    text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# begin initializing HF items, need auth token for these
hf_auth = ''
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [ ]:

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
res = generate_text(text)
print(res)
print(res[0]["generated_text"])

In [ ]:
text = """
You are a natural language understanding expert.
Your task is to classify intent and extract slots from a text.

Below are intents and explanations.
- ask_weather: ask for general weather information
- ask_temperature: ask for temperature information
- check_hot: check if the weather is hot
- check_warm: check if the weather is warm
- check_cold: check if the weather is cold
- check_cool: check if the weather is cool
- ask_weather_index: ask other weather index
- ask_climate_phenomenon: ask if a specifc climate phenomenon happens, e.g., cloudy, foggy, hail, icy, rain, sand_storm, smog, snow, sunny, windy, frosty


Below are slots and possible values of slots.
- datetime. Possbile values include:
    - morning
    - afternoon
    - night
    - today morning
    - tomorrow afternoon
    - this evening
    - 3 AM
    - 5 PM
    - next 7 days
    - next 4 hours
    - this week
    - Wednesday
- climatic_event. Possbile values include:
    - cloudy
    - foggy
    - hail
    - icy
    - rain
    - sand_storm
    - smog
    - snow
    - sunny
    - windy
    - frosty
- city. Possbile values include:
    - Berlin
    - London
    - Paris
- sun_rise_set. Possible values include:
    - sunrise
    - sunset
- relative_location. Possible values include:
    - here
    - there
    - destination
    - my destination

Based on the information above, please classify intent and extract slots from the following text.
Return the result in a json format. Do not return anything other than the json.

Below is an example.
Text: Weather Berlin next 7 days
Result:
{
    intent: ask_weather
    slot: {
        datetime: next 7 days
    }
}

Now, solve the task.
Text: what is the weather look like tomorrow evening in Munich

Return the result in a json format.

"""

In [ ]:
res = generate_text(text)
print(res)
print(res[0]["generated_text"])

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

In [ ]:
res = generate_text("Tell me the history of Elon Musk")
print(res[0]["generated_text"])